# Vehicle Classification

## Evaluate your model


In [1]:
import os
from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50


## Experiment settings

In [2]:
CONFIG_YML = "../experiments/exp_010/config.yml"
WEIGHTS = "../experiments/exp_010/model.49-1.1983.h5"

In [3]:
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v2/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 64},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.5,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.2,
    'fill_mode': 'constant',
    'fill_value': 125},
   'random_zoom': {'height_factor': 0.2, 'width_factor': 0.2},
   'random_contrast': {'factor': 0.2}}},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.0001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 50,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_010/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp_010/logs'}}}}

In [4]:
MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

In [5]:
cnn_model = resnet_50.create_model(weights = WEIGHTS)

print(cnn_model.summary())

2022-06-27 17:55:14.391300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 17:55:14.400096: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 17:55:14.400969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 17:55:14.402867: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

## Get predictions from testing dataset

In [6]:
predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


2022-06-27 17:55:23.694491: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-06-27 17:55:23.969219: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-27 17:55:23.969722: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-27 17:55:23.969769: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-06-27 17:55:23.970367: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-27 17:55:23.970455: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


In [7]:
print(classification_report(y_true=labels, y_pred=predictions))

                                                        precision    recall  f1-score   support

                            AM_General_Hummer_SUV_2000       0.70      0.95      0.81        44
                             Acura_Integra_Type_R_2001       0.97      0.66      0.78        44
                                   Acura_RL_Sedan_2012       0.70      0.66      0.68        32
                                   Acura_TL_Sedan_2012       0.83      0.67      0.74        43
                                  Acura_TL_Type-S_2008       1.00      0.90      0.95        42
                                  Acura_TSX_Sedan_2012       1.00      0.57      0.73        40
                              Acura_ZDX_Hatchback_2012       0.45      0.87      0.59        39
              Aston_Martin_V8_Vantage_Convertible_2012       0.51      0.64      0.57        45
                    Aston_Martin_V8_Vantage_Coupe_2012       0.89      0.20      0.32        41
                  Aston_Martin_Virage_C

In [8]:
acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.7416!
